In [340]:
import pandas as pd
import re
import geonamescache
import unidecode


gc = geonamescache.GeonamesCache()

gc_cities = gc.get_cities()


#all_cities = {'miami': {'12341234': {'country': 'US'},
#                        '56234234': {'country': 'RU'}},
#              'madison':{'981312' : {'Ccountry: 'US'}}}

all_cities = {}
for city_id in gc_cities:
    city_name = gc_cities[city_id]['name']
    unaccented_cityname = unidecode.unidecode(city_name)
    if unaccented_cityname.lower() not in all_cities:
        all_cities[unaccented_cityname.lower()] = {}
    all_cities[unaccented_cityname.lower()][city_id] = {'country': gc_cities[city_id]['countrycode']}


def extract_city(hl):
    hllist = hl.split(' ')
    cities = []
    possible_city_names = []
    
    for i in range(len(hllist)-3):
        possible_city_names.append(hllist[i] + ' ' + hllist[i+1] + ' ' + hllist[i+2]+ ' ' + hllist[i+3])
    
    for i in range(len(hllist)-2):
        possible_city_names.append(hllist[i] + ' ' + hllist[i+1] + ' ' + hllist[i+2])
        
    for i in range(len(hllist)-1):
        possible_city_names.append(hllist[i] + ' ' + hllist[i+1])    
   
    for i in range(len(hllist)):
        possible_city_names.append(hllist[i])

    
    #print(possible_city_names)
    for w in possible_city_names:
        if w.lower() in all_cities and w.lower()!= 'of':
                return w
    return None

    
def extract_country(hl):
    hllist = hl.split(' ')
    possible_countries = []
    
    for i in range(len(hllist)):
        possible_countries.append(hllist[i])
        
    for i in range(len(hllist)-1):
        possible_countries.append(hllist[i] + ' ' + hllist[i+1])
    
    for i in range(len(hllist)-2):
        possible_countries.append(hllist[i] + ' ' + hllist[i+1] + " " + hllist[i+2])

    for c in possible_countries:
        if c in gc.get_countries_by_names():
                return c

file = open("data/headlines.txt", 'r')
headlines = []
data = []

columns = ['headline', 'countries', 'cities']

for line in file.readlines():
    headlines.append(line.strip()) 
    regexspace = re.compile(r"(\'|\’)")
    new = regexspace.sub(' ', line)
    regexdelete = re.compile(r"(,|\?|\!)")
    then = regexdelete.sub('', new)

    city =  extract_city(then.strip())
    country = extract_country(then.strip())
    data.append([line.strip(), country, city])

    
    
df = pd.DataFrame(data, columns = columns)

df.fillna(value = pd.np.nan, inplace = True)

print(df)




                                              headline countries  \
0                             Zika Outbreak Hits Miami       NaN   
1                      Could Zika Reach New York City?       NaN   
2                    First Case of Zika in Miami Beach       NaN   
3              Mystery Virus Spreads in Recife, Brazil    Brazil   
4              Dallas man comes down with case of Zika       NaN   
..                                                 ...       ...   
645  Rumors about Rabies spreading in Jerusalem hav...       NaN   
646              More Zika patients reported in Indang       NaN   
647  Suva authorities confirmed the spread of Rotav...       NaN   
648         More Zika patients reported in Bella Vista       NaN   
649                     Zika Outbreak in Wichita Falls       NaN   

            cities  
0            Miami  
1    New York City  
2      Miami Beach  
3           Recife  
4           Dallas  
..             ...  
645      Jerusalem  
646         Ind